In [1]:
import copy
import operator
import os
import random
import warnings
from glob import glob
from os.path import join

import cv2
import matplotlib as mpl
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import pydicom
import SimpleITK as sitk
from scipy.ndimage.interpolation import zoom
from tqdm import tqdm

%matplotlib inline

In [2]:
data_root = '/data/DeepStomach/dch/20200805/'
npy_data_root = '/root/workspace/DCH_AI/data_crc_3d'

dirs_list = [
    dirname for dirname in os.listdir(data_root)
    if (not dirname.startswith('.')) and (
        os.path.isdir(join(data_root, dirname)))
]
# 在启示帧和终止帧前后向外扩展的帧数
margin = 0

In [3]:
def resample(imgs, spacing, new_spacing, order=1, output=None):
    """
    重采样，即采用新分辨率
    :param imgs:
    :param spacing:
    :param new_spacing:
    :param order:
    :return:
    """
    assert len(spacing) == len(new_spacing), 'Wrong shape'
    resize_factor = spacing / new_spacing
    # zoom 将图片每一维以相应系数缩小
    imgs = zoom(imgs,
                resize_factor,
                output=output,
                mode='nearest',
                order=order)
    return imgs


def HUTransform(img, win_width, win_loc):
    """
    灰度标准化（HU值），将HU值（[HU_min, HU_max]）线性变换至0~1内的灰度值
    :param img:
    :return:
    """
    HU_min = int(win_loc - win_width / 2.0 + 0.5)
    HU_max = int(win_loc + win_width / 2.0 + 0.5)
    HU = np.array([HU_min, HU_max]).astype('float')
    normalized_img = (img - HU_min) / (HU_max - HU_min)
    normalized_img[normalized_img < 0] = 0
    normalized_img[normalized_img > 1] = 1
    normalized_img = (normalized_img * 255).astype('uint8')
    return normalized_img


def dcm2array(dcms_dir):
    reader = sitk.ImageSeriesReader()
    dicom_names = reader.GetGDCMSeriesFileNames(dcms_dir)
    reader.SetFileNames(dicom_names)
    image3D = reader.Execute()
    # spacing顺序为[z, y ,x]
    spacing = np.array(list(reversed(image3D.GetSpacing())))
    img = sitk.GetArrayFromImage(image3D)
    return img, spacing

In [4]:
def dcm2npy(dirs, npy_data_root):
    records = []
    for dir_name in tqdm(dirs):
        dir_path = join(data_root, dir_name)
        items = [
            item_name for item_name in os.listdir(dir_path)
            if (not item_name.startswith('.')) and (
                os.path.isdir(join(dir_path, item_name)))
        ]

        stage_set = set([item_name.split('-')[1] for item_name in items])

        saved_npy_path = join(npy_data_root, dir_name)
        if not os.path.exists(saved_npy_path):
            os.makedirs(saved_npy_path)

        for stage in stage_set:
            img_saved_name = '{}_{}_crc_image.npy'.format(
                dir_name, stage.lower())
            img_saved_path = join(saved_npy_path, img_saved_name)
            label_saved_name = '{}_{}_crc_label.npy'.format(
                dir_name, stage.lower())
            label_saved_path = join(saved_npy_path, label_saved_name)
            attr_saved_name = '{}_{}_crc_attr.npy'.format(
                dir_name, stage.lower())
            attr_saved_path = join(saved_npy_path, attr_saved_name)
            if os.path.exists(img_saved_path):
                continue

            imgs_dir = join(dir_path, '{}-{}-Original'.format(dir_name, stage))
            labels_dir = join(dir_path, '{}-{}-Label'.format(dir_name, stage))

            # 获取img和label路径
            imgs, spacing = dcm2array(imgs_dir)
            imgs = HUTransform(imgs, 200, 40)
            resolution = np.array([1., 0.7, 0.7])
            new_imgs = resample(imgs, spacing, resolution)
            labels, _ = dcm2array(labels_dir)
            new_labels = resample(labels, spacing, resolution, output=np.int).astype('uint8')
            
            try:
                assert imgs.shape == labels.shape, f'{dir_path} 数据{imgs.shape}和标签{labels.shape}数目必须相等！'
            except:
                continue
            # 启始帧和终止帧
            start_idx = imgs.shape[0] - 1
            end_idx = 0
            for idx in range(imgs.shape[0]):
                img = imgs[idx]
                label = labels[idx]
                if (label != 0).any():
                    start_idx = min(idx, start_idx)
                    end_idx = max(idx, end_idx)
            # 获取标签范围
            zz, yy, xx = np.where(labels)
            # 标签坐标zyx的启始和终止坐标
            box = np.array([[np.min(zz), np.max(zz)], [np.min(yy),
                                                       np.max(yy)],
                            [np.min(xx), np.max(xx)]])
            # 增广边缘
            extendbox = np.vstack([
                np.max([[0, 0, 0], box[:, 0] - margin], axis=0),
                np.min([imgs.shape, box[:, 1] + margin], axis=0)
            ]).T.astype('int')
            # 截取原始图像
            cut_imgs = imgs[extendbox[0, 0]:extendbox[0, 1],
                            extendbox[1, 0]:extendbox[1, 1],
                            extendbox[2, 0]:extendbox[2, 1]]
            # 截取标签
            cut_labels = labels[extendbox[0, 0]:extendbox[0, 1],
                                extendbox[1, 0]:extendbox[1, 1],
                                extendbox[2, 0]:extendbox[2, 1]]
            # 相关属性
            attribute = {
                "spacing": spacing,
                "extendbox": extendbox,
                "order": "zyx"
            }
            # 存储
            np.save(img_saved_path, cut_imgs)
            np.save(label_saved_path, cut_labels)
            np.save(attr_saved_path, attribute)

In [5]:
dcm2npy(dirs_list, npy_data_root)

100%|██████████| 440/440 [01:44<00:00,  4.21it/s] 


In [6]:
# crc_labels = pd.read_excel('/data/DeepStomach/dch/20200805/CRC_list.xlsx')

# test_sample_data = pd.read_csv('../records/test_sample_data.csv',
#                                names=['sample_id'])

# test_crc_labels = test_sample_data.merge(crc_labels,
#                                          how='inner',
#                                          left_on='sample_id',
#                                          right_on='登记号')
# test_crc_labels = test_crc_labels.drop(columns='登记号').rename(
#     columns={'转移': 'label'})
# test_crc_labels.to_csv('../records/test_crc_labels.csv',
#                        index=False,
#                        header=False)

# train_sample_data = pd.read_csv('../records/train_sample_data.csv',
#                                 names=['sample_id'])

# train_crc_labels = train_sample_data.merge(crc_labels,
#                                            how='inner',
#                                            left_on='sample_id',
#                                            right_on='登记号')
# train_crc_labels = train_crc_labels.drop(columns='登记号').rename(
#     columns={'转移': 'label'})
# train_crc_labels.to_csv('../records/train_crc_labels.csv',
#                         index=False,
#                         header=False)